Goal of analysis script is to pull in excel files with manually entered intake and weight data, determine daily intake per body weight (for both water and ethanol) for subsequent analysis


Import numpy

In [12]:
import pandas as pd
import re

Open dialog box and prompt user to select file containing the data

In [13]:
from tkinter import filedialog
from tkinter import *
 
root = Tk()
root.filename =  filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("jpeg files","*.jpg"),("all files","*.*")))
print (root.filename)

C:/Users/Schindler/Documents/ProgrammingFun/Scripts/iPythonScripts/TwoBottleChoice/Intake/111.xlsx


Generate pandas dataframe from excel file

In [14]:
data = pd.read_excel(root.filename)
data_table = pd.DataFrame(data = data)

data_table.head()

,Day,Weight,EtOH position,EtOH wt,H2O rt wt,H2O lt wt,EtOH wt.1,H2O rt wt.1,H2O lt wt.1
0,Friday,30,X,0,70,70,X,X,X
1,Sunday 3%,30,R,60,65,65,X,X,X
2,Monday,30,X,59,63,63,X,X,X
3,Tuesday 6%,30,L,59,61,61,X,X,X
4,Wednesday,30,X,57,60,60,X,X,X


Get animal number from path name (animal number is file name), create array of animal number the length of the data_table so can add this as a column

In [15]:
name = root.filename
name = re.search("\d\d\d", name).group(0)
name = [name] * len(data_table)

Add animal number array as a column to data table. Add empty columns for the intake and preference data we will compute in the next step.

In [16]:
data_table = data_table.assign(Animal = name, H2O = " ", H2O_pref = " ", H2O8 = " ", EtOH = " ", EtOH_pref = " ", EtOH8 = " ", EtOH8_pref = " ")
data_table.head()

,Day,Weight,EtOH position,EtOH wt,H2O rt wt,H2O lt wt,EtOH wt.1,H2O rt wt.1,H2O lt wt.1,Animal,EtOH,EtOH8,EtOH8_pref,EtOH_pref,H2O,H2O8,H2O_pref
0,Friday,30,X,0,70,70,X,X,X,111,,,,,,,
1,Sunday 3%,30,R,60,65,65,X,X,X,111,,,,,,,
2,Monday,30,X,59,63,63,X,X,X,111,,,,,,,
3,Tuesday 6%,30,L,59,61,61,X,X,X,111,,,,,,,
4,Wednesday,30,X,57,60,60,X,X,X,111,,,,,,,


In [17]:
def I2BC(data):
    "Analyzes intermittent two bottle choice data"
    
    #Intake on EtOH days
    EtOH = []
    #EtOH pref on EtOH days
    EtOH_pref = []
    #EtOH intake on days with 8pm measurement
    EtOH8 = []
    #EtOH pref on days with 8pm measurement
    EtOH_pref8 = []
    #Water intake
    water = []
    #Water preference (for bottle on the right) on days with no EtOH
    water_pref = []
    #Water intake on days with 8pm measurement
    water8 = []
    
    i = 0
    
    for index, row in data.iterrows():
        #last row will not have intake data
        if index == len(data) - 1:
            EtOH.append(None)
            EtOH_pref.append(None)
            EtOH8.append(None)
            EtOH_pref8.append(None)
            water.append(None)
            water_pref.append(None)
            water8.append(None)
        
            i = i + 1
        
        #compute intake in grams consumed / body weight in kg
        #EtOH is 20% so * 0.2 to get actual intake in grams
        #weight recorded is in gram so / by 1000 to get kg
        
        elif row[2] == 'L':
            e = ((float(data.loc[[i],['EtOH wt']].values - data.loc[[i+1],['EtOH wt']].values)) * 0.2) / ((float(data.loc[[i],['Weight']].values)) / 1000)
            EtOH.append(e)
            
            w = (float(data.loc[[i],['H2O rt wt']].values - data.loc[[i+1],['H2O rt wt']].values)) / ((float(data.loc[[i],['Weight']].values)) / 1000)
            water.append(w)    
        
            p = e/w
            EtOH_pref.append(p)  
        
            water_pref.append(None)
            
            #compute 8pm data if collected
            if row[6] != 'X':
                e = ((float(data.loc[[i],['EtOH wt']].values - data.loc[[i],['EtOH wt.1']].values)) * 0.2) / ((float(data.loc[[i],['Weight']].values)) / 1000)
                EtOH8.append(e)
            
                w = (float(data.loc[[i],['H2O rt wt']].values - data.loc[[i],['H2O rt wt.1']].values)) / ((float(data.loc[[i],['Weight']].values)) / 1000)
                water8.append(w)    
        
                p = e/w
                EtOH_pref8.append(p)
                
            #if no 8pm data
            else:
                EtOH8.append(None)
                EtOH_pref8.append(None)
                water8.append(None)
                
            i = i + 1
        
        elif row[2] == 'R':
            e = ((float(data.loc[[i],['EtOH wt']].values - data.loc[[i+1],['EtOH wt']].values)) * 0.2) / ((float(data.loc[[i],['Weight']].values)) / 1000)
            EtOH.append(e)
            
            w = (float(data.loc[[i],['H2O lt wt']].values - data.loc[[i+1],['H2O lt wt']].values)) / ((float(data.loc[[i],['Weight']].values)) / 1000)
            water.append(w)    
        
            p = e/w
            EtOH_pref.append(p)  
        
            water_pref.append(None)
            
            if row[6] != 'X':
                e = ((float(data.loc[[i],['EtOH wt']].values - data.loc[[i],['EtOH wt.1']].values)) * 0.2) / ((float(data.loc[[i],['Weight']].values)) / 1000)
                EtOH8.append(e)
            
                w = (float(data.loc[[i],['H2O lt wt']].values - data.loc[[i],['H2O lt wt.1']].values)) / ((float(data.loc[[i],['Weight']].values)) / 1000)
                water8.append(w)    
        
                p = e/w
                EtOH_pref8.append(p)
                
            else:
                EtOH8.append(None)
                EtOH_pref8.append(None)
                water8.append(None)
                
            i = i + 1
        
        elif row[2] == 'X':
            r = float(data.loc[[i],['H2O rt wt']].values - data.loc[[i+1],['H2O rt wt']].values) / (float(data.loc[[i],['Weight']].values) / 1000)
            l = float(data.loc[[i],['H2O lt wt']].values - data.loc[[i+1],['H2O lt wt']].values) / (float(data.loc[[i],['Weight']].values) / 1000)
            w = r + l
            water.append(w)
        
            p = r/l
            water_pref.append(p)
        
            EtOH.append(None)
            EtOH_pref.append(None)
            EtOH8.append(None)
            EtOH_pref8.append(None)
            water8.append(None)
            
            i = i + 1

    data['EtOH'] = EtOH
    data['H20'] = water
    data['EtOH_pref'] = EtOH_pref
    data['H20_pref'] = water_pref
    data['EtOH8'] = EtOH8
    data['EtOH8_pref'] = EtOH_pref8
    data['H208'] = water8
    
    return data

Use the intake function on the data table

In [18]:
intake_data = data_table.pipe(I2BC)
intake_data.head()

,Day,Weight,EtOH position,EtOH wt,H2O rt wt,H2O lt wt,EtOH wt.1,H2O rt wt.1,H2O lt wt.1,Animal,EtOH,EtOH8,EtOH8_pref,EtOH_pref,H2O,H2O8,H2O_pref,H20,H20_pref,H208
0,Friday,30,X,0,70,70,X,X,X,111,NaN,NaN,NaN,NaN,,,,333.333333,1.0,NaN
1,Sunday 3%,30,R,60,65,65,X,X,X,111,6.666667,NaN,NaN,0.1,,,,66.666667,NaN,NaN
2,Monday,30,X,59,63,63,X,X,X,111,NaN,NaN,NaN,NaN,,,,133.333333,1.0,NaN
3,Tuesday 6%,30,L,59,61,61,X,X,X,111,13.333333,NaN,NaN,0.4,,,,33.333333,NaN,NaN
4,Wednesday,30,X,57,60,60,X,X,X,111,NaN,NaN,NaN,NaN,,,,66.666667,1.0,NaN
